In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/igneural1024newfeats"))
print(os.listdir("../input/egneural512newfeats"))


# Any results you write to the current directory are saved as output.

In [ ]:
igPred=pd.read_csv('../input/igneural1024newfeats/subm_0.028309_2018-12-14-21-37.csv')

In [ ]:
egPred=pd.read_csv('../input/egneural512newfeats/subm_0.836138_2018-12-14-22-54.csv')

In [ ]:
print(igPred.shape)
print(egPred.shape)

In [ ]:
for cindex in igPred.columns:
    if cindex not in egPred.columns:
        egPred[cindex]=0
        
for cindex in egPred.columns:
    if cindex not in igPred.columns:
        igPred[cindex]=0
        

In [ ]:
igPred.describe()

In [ ]:
egPred.describe()

In [ ]:
pdf=pd.concat([igPred, egPred], sort=True)


In [ ]:
pdf.shape

In [ ]:
pdf=pdf.fillna(0)

In [ ]:
pdf.describe()


In [ ]:
#from Scirpus discussion:

def GenUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

y = pd.DataFrame()
y['mymean'] = pdf[feats].mean(axis=1)
y['mymedian'] = pdf[feats].median(axis=1)
y['mymax'] = pdf[feats].max(axis=1)

pdf['class_99'] = GenUnknown(y)
pdf.describe()
pdf.to_csv('mergeNeuralNTPost99.csv', index=False)

In [ ]:
import copy
meta=pd.read_csv('../input/PLAsTiCC-2018/test_set_metadata.csv')
meta.describe()

def modUnknown(opdf, meta, ddfMult=0.5, mwMult=0.5, preserveMed=False):
    pdf=copy.deepcopy(opdf)
    mdf=pdf.merge(meta,on='object_id')
    ddfilter=mdf.loc[:,'ddf']==1
    mwfilter=mdf.loc[:,'hostgal_photoz']==0
    print(ddfilter.sum())
    print(mwfilter.sum())
    
    mdf.loc[mwfilter,'class_99']=mwMult*mdf.loc[mwfilter,'class_99']
    mdf.loc[ddfilter,'class_99']=ddfMult*mdf.loc[ddfilter,'class_99']
    pdf.loc[:,'class_99']=mdf.loc[:,'class_99']
    
    return pdf

npdf=modUnknown(pdf, meta)
npdf.describe()
pdf.to_csv('mergeNeuralNTPost99igless.csv', index=False)


In [ ]:
npdf.to_csv('mergeNeuralNTPost99.csv', index=False)